In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from imripy import merger_system as ms
from imripy import halo
from imripy import inspiral
from imripy import waveform
from imripy import detector

In [ ]:
m1 = 1e3 * ms.solar_mass_to_pc
m2 = 1. * ms.solar_mass_to_pc
D = 5e8

sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)

In [ ]:
inspiral.Classic.ln_Lambda = -1
rho_spike = 226. * ms.solar_mass_to_pc
r_spike = 0.54
e0 = 1e-4
a0 = 100 * sp_0.r_isco()
afin = 1. * sp_0.r_isco()

sp_dm = ms.SystemProp(m1, m2, halo.Spike(rho_spike, r_spike, 7./3.), D)

ev_0 = inspiral.Classic.evolve_elliptic_binary(sp_0, a0, e0, a_fin=afin, accretion=True, verbose=2, acc=1e-10)
ev_nacc = inspiral.Classic.evolve_elliptic_binary(sp_dm, a0, e0, a_fin=afin, accretion=False, verbose=2, acc=1e-10)
ev_acc = inspiral.Classic.evolve_elliptic_binary(sp_dm, a0, e0, a_fin=afin, accretion=True, verbose=2, acc=1e-10)

In [ ]:
fig, (ax_p, ax_e, ax_m, ax_E, ax_L) = plt.subplots(5, 1, figsize=(16,30))
ax_p.loglog(ev_0.t/ms.year_to_pc, ev_0.a*(1.-ev_0.e**2)/sp_0.m1, label='no dm')
ax_p.loglog(ev_nacc.t/ms.year_to_pc, ev_nacc.a*(1.-ev_nacc.a**2)/sp_dm.m1, label='nacc')
ax_p.loglog(ev_acc.t/ms.year_to_pc, ev_acc.a*(1.-ev_acc.e**2)/sp_dm.m1, label='acc')

ax_e.loglog(ev_0.t/ms.year_to_pc, ev_0.e, label='no dm')
ax_e.loglog(ev_nacc.t/ms.year_to_pc, ev_nacc.e, label='nacc')
ax_e.loglog(ev_acc.t/ms.year_to_pc, ev_acc.e, label='acc')

ax_m.loglog(ev_0.t/ms.year_to_pc, ev_0.m2/m2, label='no dm')
ax_m.loglog(ev_acc.t/ms.year_to_pc, ev_acc.m2/m2, label='acc')

ax_E.loglog(ev_0.t/ms.year_to_pc, -inspiral.Classic.E_orbit(sp_0, ev_0.a, ev_0.e), label='no dm')
ax_E.loglog(ev_nacc.t/ms.year_to_pc, -inspiral.Classic.E_orbit(sp_dm, ev_nacc.a, ev_nacc.e), label='nacc')

ax_L.loglog(ev_0.t/ms.year_to_pc, inspiral.Classic.L_orbit(sp_0, ev_0.a, ev_0.e), label='no dm')
ax_L.loglog(ev_nacc.t/ms.year_to_pc, inspiral.Classic.L_orbit(sp_dm, ev_nacc.a, ev_nacc.e), label='nacc') 

sp_dm.m2 = ev_acc.m2
ax_E.loglog(ev_acc.t/ms.year_to_pc, -inspiral.Classic.E_orbit(sp_dm, ev_acc.a, ev_acc.e), label='acc')
ax_L.loglog(ev_acc.t/ms.year_to_pc, inspiral.Classic.L_orbit(sp_dm, ev_acc.a, ev_acc.e), label='nacc')             
sp_dm.m2 = m2

ax_p.grid(); ax_p.set_xlabel('t / yr'); ax_p.set_ylabel(r'p / $GM/c^4$')
ax_p.legend();
ax_e.grid(); ax_e.set_xlabel('t / yr'); ax_e.set_ylabel('e'); 
ax_e.set_xscale('log'); ax_e.set_ylim(bottom=e0/10.);
ax_m.grid(); ax_m.set_xlabel('t / yr'); ax_m.set_ylabel(r'$\Delta m/m$') 

ax_E.grid(); ax_E.set_xlabel('t / yr'); ax_E.set_ylabel(r'-E'); ax_E.set_xscale('log')
ax_L.grid(); ax_L.set_xlabel('t / yr'); ax_L.set_ylabel(r'L'); ax_L.set_xscale('log')

In [ ]:
# Plot observable phase difference in 2nd harmonic
wf_0 = waveform.h_n(2, sp_0, ev_0, dbg=True, acc=1e-13)
wf_acc = waveform.h_n(2, sp_dm, ev_acc, dbg=True, acc=1e-13)
wf_nacc = waveform.h_n(2, sp_dm, ev_nacc, dbg=True, acc=1e-13)
from scipy.interpolate import interp1d
PhiT_0int = interp1d(wf_0[0], wf_0[4], kind='cubic', bounds_error=False, fill_value=(0.,0.))
plt.figure(figsize=(16, 10))
#plt.plot( wf_0[0]/ms.hz_to_invpc, wf_0[4], label=r"$\Phi_{0}$")
#plt.plot( wf_acc[0]/ms.hz_to_invpc, wf_acc[4], label=r"$\Phi_{acc}$")
#plt.plot( wf_nacc[0]/ms.hz_to_invpc, wf_nacc[4], label=r"$\Phi_{nacc}$")
plt.plot( wf_acc[0]/ms.hz_to_invpc, np.abs(wf_acc[4] - PhiT_0int(wf_acc[0])), label=r"$\Delta\Phi_{acc}$")
plt.plot(wf_nacc[0]/ms.hz_to_invpc, np.abs(wf_nacc[4] - PhiT_0int(wf_nacc[0])), label=r"$\Delta\Phi_{nacc}$")
PhiT_accint = interp1d(wf_acc[0], wf_acc[4], kind='cubic', bounds_error=False, fill_value=(0.,0.))
plt.plot(wf_nacc[0]/ms.hz_to_invpc, np.abs(wf_nacc[4] - PhiT_accint(wf_nacc[0])), label=r"$\delta\Phi_{acc}$")
plt.yscale('log'); plt.ylim(bottom=1e-2)
plt.xlabel('f / Hz'); #plt.xlim(right=wf_acc[0][-1]//ms.hz_to_invpc)#plt.xlim((0.01,0.1))
plt.xscale('log')
plt.grid(); plt.legend(); plt.show()

In [ ]:
def plotGWspectrum(sp, ev, ax_f, ax_Q=None, n_comp=20, n_disp = 5):
    wfs = [waveform.h_n(n, sp, ev, acc=1e-13) for n in range(1, n_comp+1)]
    for i in range( min(len(wfs), n_disp)):
        ax_f.loglog(wfs[i][0]/ms.hz_to_invpc, 2.*wfs[i][0]*np.abs(wfs[i][1]), label=r"$|h^{(" + str(i+1) + ")}_+|$")

    f_gw = np.geomspace(np.min(wfs[0][0]), np.max(wfs[n_disp][0]), 5000)

    h_plus_tot = np.sum([ np.interp(f_gw, wf[0], wf[1], left=0., right=0.) * np.exp(1.j * np.interp(f_gw, wf[0], wf[3], left=0., right=0.)) for wf in wfs ],  axis=0)
    h_cross_tot = np.sum([ np.interp(f_gw, wf[0], wf[2], left=0., right=0.) * np.exp(1.j * np.interp(f_gw, wf[0], wf[3], left=0., right=0.)) for wf in wfs ],  axis=0)
    ax_f.plot(f_gw/ms.hz_to_invpc, 2.*f_gw*np.abs(h_plus_tot), label=r"$|h^{SPA}_+|$")

    if not ax_Q is None:
        t_plot = np.linspace(np.min(ev.t) if ev.t[0] > 0. else ev.t[1]*1e-1, np.max(ev.t), 400)
        if ax_Q.get_yscale() == 'log':
            f_plot = np.geomspace(np.min(f_gw), np.max(f_gw)/1e3, 400)
        else:
            f_plot = np.linspace(np.min(f_gw), np.max(f_gw)/1e2, 400)
        t_plot, f_plot = np.meshgrid(t_plot, f_plot)

        h_plus_plot = np.zeros(shape=np.shape(t_plot)); h_cross_plot = np.zeros(shape=np.shape(t_plot))
        for i in range(len(t_plot[0])):
            for wf in wfs:
                f = np.interp(t_plot[0, i], ev.t, wf[0], left=0., right=0.)
                index_f = (np.abs(f_plot[:, i] - f)).argmin()
                h_plus_plot[index_f, i] = np.abs(np.interp(f_plot[index_f, i], wf[0], wf[1]))
                h_cross_plot[index_f, i] = np.abs(np.interp(f_plot[index_f, i], wf[0], wf[2]))

        h_plus_plot = h_plus_plot/np.max(h_plus_plot);  h_cross_plot = h_cross_plot/np.max(h_cross_plot)
        return ax_Q.contourf( t_plot/ms.year_to_pc, f_plot/ms.hz_to_invpc, h_plus_plot, cmap=plt.get_cmap("YlOrRd"))
        #ax_Q.contourf( t_plot/ms.s_to_pc, f_plot/ms.hz_to_invpc, h_cross_plot)



In [ ]:
fig, (ax_f, ax_Q) = plt.subplots(2, 1, figsize=(16,20))
mappable = plotGWspectrum(sp_0, ev_0, ax_f, ax_Q=ax_Q)
f = np.geomspace(detector.Lisa().Bandwith()[0], detector.Lisa().Bandwith()[1], 100)
ax_f.plot(f/ms.hz_to_invpc, detector.Lisa().NoiseStrain(f), label='LISA')
ax_f.grid(); ax_f.legend()
fig.colorbar(mappable)

fig, (ax_f, ax_Q) = plt.subplots(2, 1, figsize=(16,20))
mappable = plotGWspectrum(sp_dm, ev_acc, ax_f, ax_Q=ax_Q)
f = np.geomspace(detector.Lisa().Bandwith()[0], detector.Lisa().Bandwith()[1], 100)
ax_f.plot(f/ms.hz_to_invpc, detector.Lisa().NoiseStrain(f), label='LISA')
ax_f.grid(); ax_f.legend()
fig.colorbar(mappable)

fig, (ax_f, ax_Q) = plt.subplots(2, 1, figsize=(16,20))
mappable = plotGWspectrum(sp_dm, ev_nacc, ax_f, ax_Q=ax_Q)
f = np.geomspace(detector.Lisa().Bandwith()[0], detector.Lisa().Bandwith()[1], 100)
ax_f.plot(f/ms.hz_to_invpc, detector.Lisa().NoiseStrain(f), label='LISA')
ax_f.grid(); ax_f.legend()
fig.colorbar(mappable)